# Setup

In [ ]:
import snowflake.snowpark.functions as F

# set up snowpark session
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
use database SKI_GEAR_SUPPORT_DB;
use schema SKI_GEAR_SUPPORT_SCHEMA;

# Using Complete

In [ ]:
from snowflake.cortex import Complete

Complete("llama3.1-405b", "¿Cómo adquieren los copos de nieve su forma única?. Reply in english")

# System and User roles in Complete
## Options: Cortex Guard

In [ ]:
messages = [
    {'role': 'system', 'content': 'You are a helpful assistant that answers programming questions in the style of a rancher from the western United States.' },
    {'role': 'user', 'content': 'Are semicolons optional in JavaScript?'}
]

options = {
    'guardrails': True,
}

Complete("llama3.1-405b", messages, options = options)

In [ ]:
messages = [
    {'role': 'system', 'content': 'You are a helpful assistant that answers programming questions in the style of a rancher from the western United States.' },
    {'role': 'user', 'content': 'How to rob a bank?'}
]

options = {
    'guardrails': True,
}

Complete("llama3.1-405b", messages, options = options)

# Multi-turn chat

In [ ]:
messages_with_history = [
    {'role': 'system', 'content': 'You are a helpful assistant that answers programming questions in the style of a rancher from the western United States.' },
    {'role': 'user', 'content': 'Are semicolons optional in JavaScript?'},
    {'role': 'assistant', 'content': "Well, howdy there, partner! I reckon you're askin' about semicolons in JavaScript, eh? …, and may your code always be tidy!"},
    {'role': 'user', 'content': 'So to be clear, my code will not break with out them?'}
    ]

options = {
    'guardrails': True
}

from snowflake.cortex import Complete

response = Complete("llama3.1-405b", messages_with_history, options = options)

response


# Temperature

In [ ]:
options = {
    'guardrails': True,
    'temperature': 0.9
}

response = Complete("llama3.1-405b", messages_with_history, options = options)

response


# Top P

In [ ]:
options = {
    'guardrails': True,
    'top_p': 0.9
}

response = Complete("llama3.1-405b", messages_with_history, options = options)

response

# Max Tokens

In [ ]:
messages = [
    {'role': 'system', 'content': 'You are a helpful assistant that decides if a call transcript needs follow-up. You only respond by saying YES or NO.' },
    {'role': 'user', 'content': "Call Transcript: Your billing department is a disaster! I've been overcharged three times this month. Please refund me"}
]

options = {"max_tokens": 1}

response = Complete("llama3.2-1b", messages, options = options)

response

# Review Output of Complete()

In [ ]:
SELECT SNOWFLAKE.CORTEX.COMPLETE(
    'llama2-70b-chat',
    [
        {
            'role': 'system',
            'content': 'You are a helpful assistant that decides if a call transcript needs follow-up. You only respond by saying YES or NO.'
        },
        {
            'role': 'user',
            'content': transcript
        }
    ],
    {'max_tokens': 1}
) AS followup_needed
FROM CALL_TRANSCRIPTS
LIMIT 10;


# Task-Specific Functions: Translate, Sentiment, Summarize and Classify text

# Translate

In [ ]:
# Calling UnicodeTranslateError
from snowflake.cortex import Translate

article_df = session.table("CALL_TRANSCRIPTS")
article_df = article_df.filter(F.col('language')=='German')
article_df = article_df.withColumn(
    "DE_TRANSCRIPT",
    Translate(F.col('EN_TRANSCRIPT'), "en", "de")
)
article_df

In [ ]:
SELECT TRANSCRIPT, 
SNOWFLAKE.CORTEX.TRANSLATE(TRANSCRIPT, '', 'en') AS ENGLISH_TRANSCRIPTS
FROM CALL_TRANSCRIPTS 
LIMIT 10;

# Sentiment

In [ ]:
# Using Sentiment
from snowflake.cortex import Sentiment
from snowflake.snowpark.functions import col

article_df = session.table("CALL_TRANSCRIPTS")
article_df = article_df.withColumn("sentiment_score_en", Sentiment(col("EN_TRANSCRIPT")))

article_df.collect()

In [ ]:
SELECT
SNOWFLAKE.CORTEX.SENTIMENT(EN_TRANSCRIPT), EN_TRANSCRIPT
FROM CALL_TRANSCRIPTS 
LIMIT 10;

# Summarize

In [ ]:
from snowflake.cortex import Summarize

text = """
    The Snowflake company was co-founded by Thierry Cruanes, Marcin Zukowski,
    and Benoit Dageville in 2012 and is headquartered in Bozeman, Montana.
"""

Summarize(text)

In [ ]:
CREATE OR REPLACE TABLE call_transcripts
AS 
SELECT *, SNOWFLAKE.CORTEX.SUMMARIZE(EN_TRANSCRIPT) as EN_CALL_SUMMARY
FROM CALL_TRANSCRIPTS;

SELECT * FROM call_transcripts LIMIT 10;

# Classify text

In [ ]:
from snowflake.cortex import ClassifyText

ClassifyText("What is the best restaurant in my city?", ["how to", "recommendations"])

In [ ]:
SELECT EN_CALL_SUMMARY,
SNOWFLAKE.CORTEX.CLASSIFY_TEXT(TRANSCRIPT, ['missing price tags','washed out color', 'Broken zipper']) AS damage_type_label
FROM call_transcripts
Limit 20;

# Using Helper Functions

# COUNT_TOKEN

In [ ]:
SELECT SNOWFLAKE.CORTEX.COUNT_TOKENS( 'llama3.1-70b', 
    'To be Jedi is to face the truth, and choose. Give off light, or darkness, Padawan. Be a candle, or the night.') 
    AS number_of_tokens;

In [ ]:
SELECT SNOWFLAKE.CORTEX.COUNT_TOKENS( 'mistral-7b', 
    'To be Jedi is to face the truth, and choose. Give off light, or darkness, Padawan. Be a candle, or the night.') 
    AS number_of_tokens;

# TRY_COMPLETE

In [ ]:
SELECT SNOWFLAKE.CORTEX.COMPLETE(
    'llama2-70b-chaty',
    [
        {
            'role': 'user',
            'content': 'how does a snowflake get its unique pattern?'
        }
    ],
    {
        'temperature': 0.7,
        'max_tokens': 10
    }
);

In [ ]:
SELECT SNOWFLAKE.CORTEX.TRY_COMPLETE(
    'llama2-70b-chaty',
    [
        {
            'role': 'user',
            'content': 'how does a snowflake get its unique pattern?'
        }
    ],
    {
        'temperature': 0.7,
        'max_tokens': 10
    }
);